<a href="https://colab.research.google.com/github/mohsenh17/jaxLearning/blob/main/flax/flaxCheckpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade flax orbax jax

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.3/419.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 MB 7.4 MB/s eta 0:00:00
  Created wheel for orbax: filename=orbax-0.1.9-py3-none-any.whl size=1493 sha256=26ae858ce1819ec45fb4de7fc6670df253a79c08cfcec075aa00979995098d1d
  Stored in directory: /root/.cache/pip/wheels/14/7a/98/b955a4db98b54317c311ee32367994ca530721c62a87ec56a7
Successfully built orbax
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.33
    Uninstalling jaxlib-0.4.33:
      Successfully uninstalled jaxlib-0.4.33
  Attempting uninstall: jax
    Found existing installation: jax 0.4.33
    Uninstalling jax-0.4.33:
      Successfully uninstalled jax-0.4.33
  Attempting uninstall: flax
    Found existing installation: flax 0.8.5
    Uninstalling flax-0.8.5:
      Successfully uninstalled flax-0.8.5


In [2]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np

from flax import nnx
import jax.numpy as jnp

from sklearn.preprocessing import MinMaxScaler

import orbax.checkpoint as ocp

import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import io



In [3]:

# Generate a dataset with 100 samples, 10 features, 5 informative, 5 redundant, and 2 classes
X, y = make_classification(n_samples=10000, n_features=10, n_informative=8,
                          n_classes=2, random_state=42)
df = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
df['Class'] = y
csv_data = df.to_csv(index=True)


In [3]:
class CustomImageDataset(Dataset):
    def __init__(self, dataset, transform=None, target_transform=None):
        completeDF = pd.read_csv(dataset, index_col=0)
        self.labels = pd.DataFrame(completeDF['Class'])
        features_raw = pd.DataFrame(completeDF.drop('Class', axis=1))
        #self.features = pd.DataFrame(completeDF.drop('Class', axis=1))
        scaler = MinMaxScaler()
        self.features = pd.DataFrame(scaler.fit_transform(features_raw))
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        features = np.array(self.features.iloc[idx, :])
        label = self.labels.iloc[idx, 0]
        if self.transform:
            features = self.transform(features)
        if self.target_transform:
            label = self.target_transform(label)
        return features, label

In [4]:
import kagglehub
path = kagglehub.dataset_download("mssmartypants/rice-type-classification")
print(path)
/content/dataset/riceClassification.csv

100%|██████████| 888k/888k [00:00<00:00, 94.2MB/s]

Extracting files...
/root/.cache/kagglehub/datasets/mssmartypants/rice-type-classification/versions/2


NameError: name 'content' is not defined

In [6]:
! ls /root/.cache/kagglehub/datasets/mssmartypants/rice-type-classification/versions/2/
! mv /root/.cache/kagglehub/datasets/mssmartypants/rice-type-classification/versions/2/riceClassification.csv dataset

riceClassification.csv


In [7]:
#dataset = CustomImageDataset(dataset=io.StringIO(csv_data))
dataset = CustomImageDataset(dataset="riceClassification.csv")
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [.9, 0.05,0.05])
data_loader = DataLoader(train_set, batch_size=4, shuffle=True)

for features, labels in data_loader:
    print("Batch of features has shape: ",features.shape)
    print("Batch of labels has shape: ", labels.shape)
    print(features)
    print(labels)
    break

Batch of features has shape:  torch.Size([4, 10])
Batch of labels has shape:  torch.Size([4])
tensor([[0.7568, 0.7503, 0.7214, 0.7574, 0.7112, 0.8087, 0.3426, 0.5589, 0.8031,
         0.3516],
        [0.8192, 0.7784, 0.7657, 0.7490, 0.7707, 0.8597, 0.6165, 0.5892, 0.8091,
         0.3420],
        [0.3573, 0.6650, 0.2552, 0.9352, 0.3369, 0.4401, 0.1183, 0.4155, 0.6118,
         0.6982],
        [0.4589, 0.6977, 0.3798, 0.8956, 0.4299, 0.5423, 0.3457, 0.4572, 0.6647,
         0.5847]], dtype=torch.float64)
tensor([0, 0, 1, 1])


In [8]:
class MLP(nnx.Module):
    def __init__(self, din, dm1, dm2, dm3, num_classes: int, rngs: nnx.Rngs):
      self.dp1 = nnx.Dropout(rate=0.4, rngs=rngs)
      self.linear1 = nnx.Linear(10, 16, rngs=rngs)
      self.bn1 = nnx.BatchNorm(16, rngs=rngs)
      self.dp2 = nnx.Dropout(rate=0.2, rngs=rngs)
      self.linear2 = nnx.Linear(16, 32, rngs=rngs)
      self.bn2 = nnx.BatchNorm(32, rngs=rngs)
      self.dp3 = nnx.Dropout(rate=0.1, rngs=rngs)
      self.linear3 = nnx.Linear(32, 16, rngs=rngs)
      self.bn3 = nnx.BatchNorm(16, rngs=rngs)
      self.linear4 = nnx.Linear(16, 1, rngs=rngs)


    def __call__(self, x):
      x = self.dp1(x)
      x = self.linear1(x)
      x = nnx.gelu(x)
      x = self.bn1(x)
      x = self.dp2(x)
      x = self.linear2(x)
      x = nnx.gelu(x)
      x = self.bn2(x)
      x = self.dp3(x)
      x = self.linear3(x)
      x = nnx.gelu(x)
      x = self.bn3(x)
      x= self.linear4(x)
      return x #nnx.sigmoid(x)

# Instantiate and test the model
model = MLP(10, 16, 32, 16, 1, rngs=nnx.Rngs(0))
y = model(x=jnp.ones((3, 10)))

nnx.display(model)

#model = MLP([2, 4], 1, rngs=nnx.Rngs(0))
y = model(x=jnp.ones((3, 10)))

nnx.display(y)
y



MLP(
  dp1=Dropout(rate=0.4, broadcast_dims=(), deterministic=False, rng_collection='dropout', rngs=Rngs(
    default=RngStream(
      key=RngKey(
        value=Array((), dtype=key<fry>) overlaying:
        [0 0],
        tag='default'
      ),
      count=RngCount(
        value=Array(17, dtype=uint32),
        tag='default'
      )
    )
  )),
  linear1=Linear(
    kernel=Param(
      value=Array(shape=(10, 16), dtype=float32)
    ),
    bias=Param(
      value=Array(shape=(16,), dtype=float32)
    ),
    in_features=10,
    out_features=16,
    use_bias=True,
    dtype=None,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    kernel_init=<function variance_scaling.<locals>.init at 0x7d17a70ff400>,
    bias_init=<function zeros at 0x7d17a7a52050>,
    dot_general=<function dot_general at 0x7d17a836ae60>
  ),
  bn1=BatchNorm(
    mean=BatchStat(
      value=Array(shape=(16,), dtype=float32)
    ),
    var=BatchStat(
      value=Array(shape=(16,), dtype=float32)
    ),

Array([[-0.19967312],
       [ 0.56698495],
       [-0.36731178]], dtype=float32)

In [10]:
import optax
learning_rate = 0.005
momentum = 0.9

model = MLP(10, 16, 32, 16, 1, rngs=nnx.Rngs(0))
optimizer = nnx.Optimizer(model, optax.adam(learning_rate))
metrics = nnx.MultiMetric(
  loss=nnx.metrics.Average('loss'),
)

In [11]:

def loss_fn(model: MLP, batch):
  logits = model(batch['features'])
  loss = optax.sigmoid_binary_cross_entropy(
    logits=logits, labels=batch['labels'].reshape(-1, 1)
  ).mean()
  #loss = (logits - batch['labels'])**2
  #print(logits.pval)
  return loss, logits

@nnx.jit
def train_step(model: MLP, optimizer: nnx.Optimizer, metrics: nnx.MultiMetric, batch):
  """Train for a single step."""
  grad_fn = nnx.value_and_grad(loss_fn, has_aux=True)
  (loss, logits), grads = grad_fn(model, batch)

  metrics.update(loss=loss, logits=logits, labels=batch['labels'])  # In-place updates.
  optimizer.update(grads)  # In-place updates.
  return loss, nnx.sigmoid(logits)

@nnx.jit
def eval_step(model: MLP, metrics: nnx.MultiMetric, batch):
  loss, logits = loss_fn(model, batch)
  metrics.update(loss=loss, logits=logits, labels=batch['labels'])  # In-place updates.
  return loss

In [12]:
def custom_collate_fn(batch):
    transposed_data = list(zip(*batch))

    labels =  np.array(transposed_data[1])
    features = np.array(transposed_data[0])

    return {"features":features, "labels":labels}

train_set, val_set, test_set = torch.utils.data.random_split(dataset, [.7, 0.1,0.2])

train_ds = DataLoader(train_set, batch_size=64, shuffle=True, drop_last=True, collate_fn=custom_collate_fn)
val_set = DataLoader(val_set, batch_size=4, shuffle=True, drop_last=True, collate_fn=custom_collate_fn)
test_ds = DataLoader(test_set, batch_size=4, shuffle=True, drop_last=True, collate_fn=custom_collate_fn)

# test
batch_data = next(iter(train_ds))
imgs = batch_data['features']
lbls = batch_data['labels']
print(imgs.shape, imgs[0].dtype, lbls.shape, lbls[0].dtype)
print(lbls)

#loss = train_step(model, optimizer, metrics, batch_data)
loss, logits = loss_fn(model, batch_data)
print(loss.shape, logits.shape)
print(f'{loss = }')
#print(f'{logits = }')
print(f'{optimizer.step.value = }')


(64, 10) float64 (64,) int64
[0 0 0 1 0 0 1 0 1 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 0 1 1 1 0 0 1
 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1 0 1 0 1 1 0 0 1 1 0 0]
() (64, 1)
loss = Array(0.71133876, dtype=float32)
optimizer.step.value = Array(0, dtype=uint32)


In [13]:
import jax
#model.train()
metrics_history = {
  'train_loss': [],
  'train_accuracy': [],
  'train_precision': [],
  'train_recall': [],
  'train_f1': [],
  'val_loss': [],
  'val_accuracy': [],
  'val_precision': [],
  'val_recall': [],
  'val_f1': [],
  'test_loss': [],
  'test_accuracy': [],
  'test_precision': [],
  'test_recall': [],
  'test_f1': [],
}

num_epochs = 40
for epoch in range(num_epochs):
    for batch in train_ds:
      loss, logits = train_step(model, optimizer, metrics, batch)

    for metric, value in metrics.compute().items():
      metrics_history[f'train_{metric}'].append(value)

    metrics.reset()
    print(
      f"[train] epoch: {epoch}, "
      f"loss: {metrics_history['train_loss'][-1]}, "
    )

[train] epoch: 0, loss: 0.3609307110309601, 
[train] epoch: 1, loss: 0.18825207650661469, 
[train] epoch: 2, loss: 0.16301099956035614, 
[train] epoch: 3, loss: 0.1410873830318451, 
[train] epoch: 4, loss: 0.13903899490833282, 
[train] epoch: 5, loss: 0.13084694743156433, 
[train] epoch: 6, loss: 0.11954084783792496, 
[train] epoch: 7, loss: 0.12038098275661469, 
[train] epoch: 8, loss: 0.11598961800336838, 
[train] epoch: 9, loss: 0.11178449541330338, 
[train] epoch: 10, loss: 0.10916774719953537, 
[train] epoch: 11, loss: 0.10636737197637558, 
[train] epoch: 12, loss: 0.10746248811483383, 
[train] epoch: 13, loss: 0.10622826218605042, 
[train] epoch: 14, loss: 0.10226988792419434, 
[train] epoch: 15, loss: 0.10102538019418716, 
[train] epoch: 16, loss: 0.10771719366312027, 
[train] epoch: 17, loss: 0.09823504090309143, 
[train] epoch: 18, loss: 0.10105704516172409, 
[train] epoch: 19, loss: 0.09536466002464294, 
[train] epoch: 20, loss: 0.09680689871311188, 
[train] epoch: 21, loss: 

In [14]:
#model.eval() # Switch to evaluation mode.

@nnx.jit
def pred_step(model: MLP, batch):
  logits = model(batch['features'])
  return nnx.sigmoid(logits)

test_ds = DataLoader(test_set, batch_size=32, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)

ypred = []
label = []
for test_batch in test_ds:
  logits = pred_step(model, test_batch)
  #print(np.ravel(logits))
  #print(np.ravel(logits))
  #break
  ypred.extend(np.ravel(logits))
  label.extend(np.ravel(test_batch["labels"]))
  #print(logits, test_batch["labels"])
  #break
print(ypred)
binary_ypred = np.where(np.array(ypred) > 0.5, 1, 0)
print(len(label), sum(label))
accuracy = sum([1 for pred, true in zip(binary_ypred, label) if pred == true]) / len(label)
print(f"Accuracy: {accuracy:.4f}")

[0.8388507, 0.6814183, 0.0045904433, 0.9608203, 0.9998684, 0.9699249, 0.997973, 0.9962095, 0.9953205, 0.0035713236, 0.9991178, 0.99937856, 0.9868825, 0.9971988, 8.94274e-06, 0.9140317, 6.76457e-05, 0.9298944, 0.8736703, 0.1722382, 0.0005041713, 1.1624091e-07, 0.99262416, 0.006653903, 0.9632338, 0.99684167, 0.99898046, 0.00094376085, 0.0021273254, 4.3147926e-07, 0.9979582, 0.98690295, 0.99979514, 9.292264e-05, 0.9984646, 0.99429953, 0.8424435, 0.99933004, 1.8321753e-06, 0.9999634, 0.99941254, 0.9885642, 0.9998124, 5.220526e-07, 0.0061407452, 0.0003635076, 0.9996294, 0.9998596, 0.99740833, 0.6951318, 0.9631061, 0.00071271765, 0.737289, 2.4510264e-05, 0.058060925, 0.9942801, 0.9987179, 0.99140996, 0.91261804, 0.0001780182, 1.2471223e-05, 0.0009101696, 0.97418857, 0.007924607, 0.99887735, 0.0007361447, 0.00025779151, 0.95175827, 0.00013428627, 8.144445e-06, 0.99879444, 0.99958926, 0.9427793, 0.9823737, 0.9158278, 0.017996624, 6.90913e-07, 0.9996954, 0.9966582, 0.99627054, 0.6429792, 0.9945

In [15]:
import jax
from orbax.checkpoint.type_handlers import TypeHandler
from orbax.checkpoint.type_handlers import register_type_handler
from copy import deepcopy

state = nnx.state(model)
_, state = nnx.split(model)
stateOrg = deepcopy(state)
nnx.display(state)
prng_key_value = state["dp1"]["rngs"]["default"]["key"].value
#prng_key_value2 = state["dp2"]["rngs"]["default"]["key"].value
#prng_key_value3 = state["dp3"]["rngs"]["default"]["key"].value
#prng_key_value = state["dp4"]["rngs"]["default"]["key"].value
#print(prng_key_value.dtype)
state["dp1"]["rngs"]["default"]["key"] = nnx.VariableState(
    type=nnx.Param, value=jax.random.key_data(prng_key_value), tag='default'
)
"""
state["dp2"]["rngs"]["default"]["key"] = nnx.VariableState(
    type=nnx.Param, value=jax.random.key_data(prng_key_value2), tag='default'
)
state["dp3"]["rngs"]["default"]["key"] = nnx.VariableState(
    type=nnx.Param, value=jax.random.key_data(prng_key_value3), tag='default'
)"""

#nnx.display(state)
# Save the parameters
ckpt_dir = ocp.test_utils.erase_and_create_empty('/content/my-checkpoints/')
checkpointer = ocp.PyTreeCheckpointer()
checkpointer.save('/content/my-checkpoints/state', state)

State({
  'bn1': {
    'bias': VariableState(
      type=Param,
      value=Array([-0.03455117,  0.26241738,  0.33922154,  0.10407713,  0.08230435,
             -0.32102835, -0.32010397,  0.15780467,  0.29116517,  0.4769199 ,
              0.14203657, -0.55824643, -0.02837778, -0.05293359,  0.49786356,
             -0.06772533], dtype=float32)
    ),
    'mean': VariableState(
      type=BatchStat,
      value=Array([-0.07490372, -0.06379531,  0.04793598, -0.07928919, -0.07841757,
              0.07537311,  0.59677255,  1.7198774 , -0.01150165,  0.08214779,
             -0.00557855,  0.20106544, -0.06757829,  0.37422734, -0.0277496 ,
              0.2827872 ], dtype=float32)
    ),
    'scale': VariableState(
      type=Param,
      value=Array([0.8666285 , 0.83494526, 1.3698127 , 0.98531693, 1.1697022 ,
             1.0743585 , 1.4047605 , 1.1715559 , 1.1336113 , 1.543907  ,
             1.2256165 , 0.9802988 , 1.0446272 , 1.2718362 , 1.2823906 ,
             0.95863837], dtype=float3

In [16]:
newModel = MLP(10, 16, 32, 16, 1, rngs=nnx.Rngs(0))
abstract_model = nnx.eval_shape(lambda: newModel)
graphdef, abstract_state = nnx.split(abstract_model)
print('The abstract NNX state (all leaves are abstract arrays):')
#nnx.display(abstract_state)

state_restored = checkpointer.restore(ckpt_dir / 'state', abstract_state)
#nnx.display(state_restored['dp1'])
prng_key_value = state["dp1"]["rngs"]["default"]["key"].value
state_restored["dp1"]["rngs"]["default"]["key"].type(nnx.RngKey)
state_restored["dp1"]["rngs"]["default"]["key"].value = jax.random.wrap_key_data(prng_key_value)
"""= nnx.VariableState(
    type=nnx.RngKey, value=jax.random.wrap_key_data(prng_key_value), tag='default'
)"""
nnx.display(state_restored['dp1'])
nnx.display(stateOrg['dp1'])
jax.tree.map(np.testing.assert_array_equal, stateOrg, state_restored)
print('NNX State restored: ')
#nnx.display(state_restored)

# The model is now good to use!
newModel = nnx.merge(graphdef, state_restored)
optimizer = nnx.Optimizer(newModel, optax.adam(learning_rate))
metrics = nnx.MultiMetric(
  loss=nnx.metrics.Average('loss'),
)


The abstract NNX state (all leaves are abstract arrays):
State({
  'rngs': {
    'default': {
      'count': VariableState(
        type=RngCount,
        value=Array(24116, dtype=uint32),
        tag='default'
      ),
      'key': VariableState(
        type=RngKey,
        value=Array((), dtype=key<fry>) overlaying:
        [0 0],
        tag='default'
      )
    }
  }
})
State({
  'rngs': {
    'default': {
      'count': VariableState(
        type=RngCount,
        value=Array(24116, dtype=uint32),
        tag='default'
      ),
      'key': VariableState(
        type=RngKey,
        value=Array((), dtype=key<fry>) overlaying:
        [0 0],
        tag='default'
      )
    }
  }
})
NNX State restored: 


/usr/local/lib/python3.10/dist-packages/orbax/checkpoint/type_handlers.py:1330: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


In [18]:
import jax

metrics_history = {
  'train_loss': [],
  'train_accuracy': [],
  'train_precision': [],
  'train_recall': [],
  'train_f1': [],
  'val_loss': [],
  'val_accuracy': [],
  'val_precision': [],
  'val_recall': [],
  'val_f1': [],
  'test_loss': [],
  'test_accuracy': [],
  'test_precision': [],
  'test_recall': [],
  'test_f1': [],
}

num_epochs = 40
for epoch in range(num_epochs):
    for batch in train_ds:
      loss, logits = train_step(newModel, optimizer, metrics, batch)

    for metric, value in metrics.compute().items():
      metrics_history[f'train_{metric}'].append(value)

    metrics.reset()
    print(
      f"[train] epoch: {epoch}, "
      f"loss: {metrics_history['train_loss'][-1]}, "
    )

[train] epoch: 0, loss: 0.7126048803329468, 
[train] epoch: 1, loss: 0.6890543699264526, 
[train] epoch: 2, loss: 0.6823462843894958, 
[train] epoch: 3, loss: 0.6804338097572327, 
[train] epoch: 4, loss: 0.6683182120323181, 
[train] epoch: 5, loss: 0.6706789135932922, 
[train] epoch: 6, loss: 0.6680969595909119, 
[train] epoch: 7, loss: 0.6665178537368774, 
[train] epoch: 8, loss: 0.6604345440864563, 
[train] epoch: 9, loss: 0.6601880192756653, 
[train] epoch: 10, loss: 0.6607267260551453, 
[train] epoch: 11, loss: 0.6510515213012695, 
[train] epoch: 12, loss: 0.652760922908783, 
[train] epoch: 13, loss: 0.6461787819862366, 
[train] epoch: 14, loss: 0.6518374085426331, 
[train] epoch: 15, loss: 0.6428118348121643, 
[train] epoch: 16, loss: 0.6472353339195251, 
[train] epoch: 17, loss: 0.6407294273376465, 
[train] epoch: 18, loss: 0.6446413993835449, 
[train] epoch: 19, loss: 0.6374732851982117, 
[train] epoch: 20, loss: 0.640681266784668, 
[train] epoch: 21, loss: 0.6344079375267029, 


In [17]:
#newModel.eval() # Switch to evaluation mode.

@nnx.jit
def pred_step(model: MLP, batch):
  logits = model(batch['features'])
  return logits

test_ds = DataLoader(test_set, batch_size=32, shuffle=False, drop_last=True, collate_fn=custom_collate_fn)

ypred = []
label = []
for test_batch in test_ds:
  logits = pred_step(newModel, test_batch)
  #print(np.ravel(logits))
  #print(np.ravel(logits))
  #break
  ypred.extend(np.ravel(logits))
  label.extend(np.ravel(test_batch["labels"]))
  #print(logits, test_batch["labels"])
  #break
binary_ypred = np.where(np.array(ypred) > 0.5, 1, 0)
print(len(label), sum(label))
accuracy = sum([1 for pred, true in zip(binary_ypred, label) if pred == true]) / len(label)
print(f"Accuracy: {accuracy:.4f}")

3616 1917
Accuracy: 0.9635
